### COMPUTE number of common electrodes

In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

th = '0.01'
freqs = ['0_theta', '1_alpha','2_beta','3_gamma']
rois_to_keep = ['ACC','Amg','Amg-PirT','HC','IFG','Ins','MFG','OFC','PHG','SFG','pPirT']
nb_elecs = [120,55,55]
###############################################################################
path = r'/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/figure/'
path_npz_enc = join(path, '0_Classif_Power_E_EpiPerf_LowHigh_1000perm_BBG/')
path_npz_ret = join(path, '0_Classif_Power_R_EpiPerf_LowHigh_1000perm_BBG/')
npz_form = 'All_subjects_sources_{}_odor_low_high_sel_physFT.npz'
mask_form = 'All_subjects_mask_stat_{}_minwin1.0_th'+th+'.npy'
path2save = join(path, 'Conjonction/')
###############################################################################

for freq in freqs:
    #select all electrodes from encoding (specific regions and subjects)
    mat_enc = np.load(path_npz_enc+npz_form.format(freq))
    idx_E = np.where([roi in rois_to_keep for roi in mat_enc['s_labels']])[0]
    id_su = np.where([su != 'S6' for su in mat_enc['su_codes'][idx_E]])[0]
    xyz0, labels0 = mat_enc['s_xyz'][idx_E][id_su], mat_enc['s_labels'][idx_E][id_su]
    mask0 = np.load(path_npz_enc+'masks_stat/'+mask_form.format(freq))[id_su]
    mask0 = np.asarray([not x for x in mask0])
        
    #select all electrodes from retrieval (specific regions, all subjects)
    mat_ret = np.load(path_npz_ret+npz_form.format(freq))
    idx_R = np.where([roi in rois_to_keep for roi in mat_ret['s_labels']])[0]
    xyz1, labels1 = mat_ret['s_xyz'][idx_R], mat_ret['s_labels'][idx_R]
    mask1 = np.load(path_npz_ret+'masks_stat/'+mask_form.format(freq))
    mask1 = np.asarray([not x for x in mask1])
    
    #select only electrodes present both during encoding and retrieval
    to_keep_enc = np.where((xyz0==xyz1[:,None]).all(-1))[1]
    to_keep_ret = np.where((xyz1==xyz0[:,None]).all(-1))[1]
    mask0_new, mask1_new = mask0[to_keep_enc], mask1[to_keep_ret]
    print(len(mask0_new),len(mask1_new))
    
    xyz_shared = xyz0[to_keep_enc]
    xyz0_sig = xyz0[to_keep_enc][mask0_new]
    xyz1_sig = xyz1[to_keep_ret][mask1_new]
    print(freq,xyz0_sig.shape,xyz1_sig.shape)
            
    #create an array of region labels for encoding and retrieval
    dict_regions ={ 'ACC':'Frontal','IFG':'Frontal','MFG':'Frontal','SFG':'Frontal',
                'Amg':'Olf','pPirT':'Olf','Amg-PirT':'Olf','Ins':'Olf','OFC':'Olf',
                'HC':'MTL','PHG':'MTL'}
    regions0 = np.vectorize(dict_regions.get)(labels0[to_keep_enc])
        
    #loop on all elecs and create a code indicating when it decodes
    codes_elecs, region_elecs = [], []
    for i, elec in enumerate(xyz_shared):
        in_1 = np.where(np.all(elec == xyz0_sig, 1))[0] #encoding
        in_2 = np.where(np.all(elec == xyz1_sig, 1))[0] #retrieval
        
        if in_1.size and in_2.size: #decode in both
            codes_elecs.append(2)
            region_elecs.append(regions0[i])
        elif in_1.size and not in_2.size: #decode only in encoding
            codes_elecs.append(0)
            region_elecs.append(regions0[i])
        elif not in_1.size and in_2.size: #decode only in retrieval
            codes_elecs.append(1)
            region_elecs.append(regions0[i])
    
    codes_elecs = np.asarray(codes_elecs)[np.newaxis]
    region_elecs = np.asarray(region_elecs)[np.newaxis]
    count = np.ones(shape=region_elecs.shape)
    #print(freq,'regions',region_elecs.shape,codes_elecs.shape, count.shape)
    concat = np.concatenate((codes_elecs,region_elecs,count),axis=0).swapaxes(0,1)
    df = pd.DataFrame(concat,columns=['codes_elecs','region_elecs','count'])
    df_gr = df.groupby(['codes_elecs','region_elecs']).count()
    df_gr = df_gr.unstack(level=0).T
    codes = df_gr.index.get_level_values(1).values
    rois = df_gr.columns.get_level_values(0).values
    print(df_gr, codes, rois)

    
    #Plot number of signif electrodes
    fig = plt.figure()
    plt.title('Signif electrodes for '+freq)
    plt.ylabel('#electrodes')
    xticks, w = np.arange(0,len(rois)), 0.8
    plt.xticks(xticks,rois)
    plt.ylim(0,85)
    #colors = ["#114693", "#DA0615", "#30169A"]
    colors = ["indigo","yellow","teal","grey"]
    legends = ['Encoding','Retrieval','Both','Not sig']
    bottom = np.zeros(len(df_gr.columns))
    for i,code in enumerate(codes):
        nb = df_gr.iloc[i,:]
        percent = [(nb[i]/nb_elecs[i])*100 for i in range(3)]
        print('percent',percent, [nb_elecs[i] for i in range(3)])
        plt.bar(xticks,percent,color=colors[i],bottom=bottom,label=legends[i])
        bottom += percent
    #print('count',freq,(df_gr.iloc[i,:]/nb_elecs[i])*100)
    plt.legend(loc='best')
    plt.savefig(path2save+'Nb_elecs_'+freq+'_signif_th_01.png')
    plt.savefig(path2save+'Nb_elecs_'+freq+'_signif_th_01.pdf')
    plt.clf()
    plt.close()


230 230
0_theta (81, 3) (45, 3)
region_elecs       Frontal  MTL  Olf
      codes_elecs                   
count 0                 29   15   23
      1                 20    6    5
      2                  7    3    4 ['0' '1' '2'] ['Frontal' 'MTL' 'Olf']
percent [24.166666666666668, 27.27272727272727, 41.818181818181813] [120, 55, 55]
percent [16.666666666666664, 10.909090909090908, 9.0909090909090917] [120, 55, 55]
percent [5.833333333333333, 5.4545454545454541, 7.2727272727272725] [120, 55, 55]
230 230
1_alpha (66, 3) (43, 3)
region_elecs       Frontal  MTL  Olf
      codes_elecs                   
count 0                 27   12   12
      1                 18    5    5
      2                  4    5    6 ['0' '1' '2'] ['Frontal' 'MTL' 'Olf']
percent [22.5, 21.818181818181817, 21.818181818181817] [120, 55, 55]
percent [15.0, 9.0909090909090917, 9.0909090909090917] [120, 55, 55]
percent [3.3333333333333335, 9.0909090909090917, 10.909090909090908] [120, 55, 55]
230 230
2_beta (88, 3)

In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

th = '0.01'
freqs = ['0_theta', '1_alpha','2_beta','3_gamma']
rois_to_keep = ['ACC','Amg','Amg-PirT','HC','IFG','Ins','MFG','OFC','PHG','SFG','pPirT']
nb_elecs = 120+55+55
###############################################################################
path = r'/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/figure/'
path_npz_enc = join(path, '0_Classif_Power_E_EpiPerf_LowHigh_1000perm_BBG/')
path_npz_ret = join(path, '0_Classif_Power_R_EpiPerf_LowHigh_1000perm_BBG/')
npz_form = 'All_subjects_sources_{}_odor_low_high_sel_physFT.npz'
mask_form = 'All_subjects_mask_stat_{}_minwin1.0_th'+th+'.npy'
path2save = join(path, 'Conjonction/')
###############################################################################

for freq in freqs:
    #select all electrodes from encoding (specific regions and subjects)
    mat_enc = np.load(path_npz_enc+npz_form.format(freq))
    idx_E = np.where([roi in rois_to_keep for roi in mat_enc['s_labels']])[0]
    id_su = np.where([su != 'S6' for su in mat_enc['su_codes'][idx_E]])[0]
    xyz0, labels0 = mat_enc['s_xyz'][idx_E][id_su], mat_enc['s_labels'][idx_E][id_su]
    mask0 = np.load(path_npz_enc+'masks_stat/'+mask_form.format(freq))[id_su]
    mask0 = np.asarray([not x for x in mask0])
        
    #select all electrodes from retrieval (specific regions, all subjects)
    mat_ret = np.load(path_npz_ret+npz_form.format(freq))
    idx_R = np.where([roi in rois_to_keep for roi in mat_ret['s_labels']])[0]
    xyz1, labels1 = mat_ret['s_xyz'][idx_R], mat_ret['s_labels'][idx_R]
    mask1 = np.load(path_npz_ret+'masks_stat/'+mask_form.format(freq))
    mask1 = np.asarray([not x for x in mask1])
    
    #select only electrodes present both during encoding and retrieval
    to_keep_enc = np.where((xyz0==xyz1[:,None]).all(-1))[1]
    to_keep_ret = np.where((xyz1==xyz0[:,None]).all(-1))[1]
    mask0_new, mask1_new = mask0[to_keep_enc], mask1[to_keep_ret]
    print(len(mask0_new),len(mask1_new))
    
    xyz_shared = xyz0[to_keep_enc]
    xyz0_sig = xyz0[to_keep_enc][mask0_new]
    xyz1_sig = xyz1[to_keep_ret][mask1_new]
    print(freq,xyz0_sig.shape,xyz1_sig.shape)
            
    #create an array of region labels for encoding and retrieval
    dict_regions ={ 'ACC':'Frontal','IFG':'Frontal','MFG':'Frontal','SFG':'Frontal',
                'Amg':'Olf','pPirT':'Olf','Amg-PirT':'Olf','Ins':'Olf','OFC':'Olf',
                'HC':'MTL','PHG':'MTL'}
    regions0 = np.vectorize(dict_regions.get)(labels0[to_keep_enc])
        
    #loop on all elecs and create a code indicating when it decodes
    codes_elecs, region_elecs, regions_both = [], [], []
    for i, elec in enumerate(xyz_shared):
        in_1 = np.where(np.all(elec == xyz0_sig, 1))[0] #encoding
        in_2 = np.where(np.all(elec == xyz1_sig, 1))[0] #retrieval
        
        if in_1.size and in_2.size: #decode in both
            codes_elecs.append(2)
            region_elecs.append(regions0[i])
            regions_both.append(regions0[i])
        elif in_1.size and not in_2.size: #decode only in encoding
            codes_elecs.append(0)
            region_elecs.append(regions0[i])
        elif not in_1.size and in_2.size: #decode only in retrieval
            codes_elecs.append(1)
            region_elecs.append(regions0[i])
    
    print(freq,regions_both)
    codes_elecs = np.asarray(codes_elecs)[np.newaxis]
    region_elecs = np.asarray(region_elecs)[np.newaxis]
    count = np.ones(shape=region_elecs.shape)
    #print(freq,'regions',region_elecs.shape,codes_elecs.shape, count.shape)
    concat = np.concatenate((codes_elecs,region_elecs,count),axis=0).swapaxes(0,1)
    df = pd.DataFrame(concat,columns=['codes_elecs','region_elecs','count'])
    df_gr = df[['codes_elecs','region_elecs']].groupby(['codes_elecs']).count()
    codes = df_gr.index.get_level_values(0).values
    rois = df_gr.columns.get_level_values(0).values
    print(df_gr, codes, rois)
    
    #Plot number of signif electrodes
    fig = plt.figure()
    plt.title('Signif electrodes for '+freq)
    plt.ylabel('#electrodes')
    xticks, w = np.arange(0,len(rois)), 0.8
    plt.xticks(xticks,rois)
    plt.ylim(0,85)
    #colors = ["#114693", "#DA0615", "#30169A"]
    colors = ["indigo","yellow","teal","grey"]
    legends = ['Encoding','Retrieval','Both','Not sig']
    bottom = np.zeros(len(rois))
    for i,code in enumerate(codes):
        nb = df_gr.iloc[i,:].values
        percent = (nb/nb_elecs)*100
        print('percent',percent)
        plt.bar(xticks,percent,color=colors[i],bottom=bottom,label=legends[i])
        bottom += percent
    #print('count',freq,(df_gr.iloc[i,:]/nb_elecs[i])*100)
    plt.legend(loc='best')
    plt.savefig(path2save+'Nb_elecs_'+freq+'_all_th_01.png')
    plt.savefig(path2save+'Nb_elecs_'+freq+'_all_th_01.pdf')
    plt.clf()
    plt.close()


230 230
0_theta (81, 3) (45, 3)
0_theta ['Frontal', 'Frontal', 'Frontal', 'Frontal', 'Frontal', 'Frontal', 'Frontal', 'MTL', 'MTL', 'Olf', 'Olf', 'Olf', 'MTL', 'Olf']
             region_elecs
codes_elecs              
0                      67
1                      31
2                      14 ['0' '1' '2'] ['region_elecs']
percent [ 29.13043478]
percent [ 13.47826087]
percent [ 6.08695652]
230 230
1_alpha (66, 3) (43, 3)
1_alpha ['Frontal', 'Frontal', 'MTL', 'MTL', 'MTL', 'Olf', 'MTL', 'MTL', 'Olf', 'Olf', 'Olf', 'Olf', 'Frontal', 'Olf', 'Frontal']
             region_elecs
codes_elecs              
0                      51
1                      28
2                      15 ['0' '1' '2'] ['region_elecs']
percent [ 22.17391304]
percent [ 12.17391304]
percent [ 6.52173913]
230 230
2_beta (88, 3) (43, 3)
2_beta ['Frontal', 'Frontal', 'Frontal', 'MTL', 'Olf', 'Olf', 'Olf', 'Olf', 'Olf', 'Frontal', 'Olf', 'MTL', 'MTL', 'Olf', 'Olf', 'Frontal']
             region_elecs
codes_elecs     

#### Stats on the number of electrodes // Common - Retrieval specific

In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from mne.stats import fdr_correction
from ttest_indep_perm import ttest_perm_unpaired

th = '0.01'
freqs = ['0_theta', '1_alpha','2_beta','3_gamma']
rois_to_keep = ['ACC','Amg','Amg-PirT','HC','IFG','Ins','MFG','OFC','PHG','SFG','pPirT']
nb_elecs = [120,55,55]
###############################################################################
path = r'/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/figure/'
path_npz_enc = join(path, '0_Classif_Power_E_EpiPerf_LowHigh_1000perm_BBG/')
path_npz_ret = join(path, '0_Classif_Power_R_EpiPerf_LowHigh_1000perm_BBG/')
npz_form = 'All_subjects_sources_{}_odor_low_high_sel_physFT.npz'
mask_form = 'All_subjects_mask_stat_{}_minwin1.0_th'+th+'.npy'
path2save = join(path, 'Conjonction/')
###############################################################################

Tvals, pvals = [], []
for freq in freqs:
    #select all electrodes from encoding (specific regions and subjects)
    mat_enc = np.load(path_npz_enc+npz_form.format(freq))
    idx_E = np.where([roi in rois_to_keep for roi in mat_enc['s_labels']])[0]
    id_su = np.where([su != 'S6' for su in mat_enc['su_codes'][idx_E]])[0]
    xyz0, labels0 = mat_enc['s_xyz'][idx_E][id_su], mat_enc['s_labels'][idx_E][id_su]
    su_codes0 = mat_enc['su_codes'][idx_E][id_su]
    mask0 = np.load(path_npz_enc+'masks_stat/'+mask_form.format(freq))[id_su]
    mask0 = np.asarray([not x for x in mask0])
        
    #select all electrodes from retrieval (specific regions, all subjects)
    mat_ret = np.load(path_npz_ret+npz_form.format(freq))
    idx_R = np.where([roi in rois_to_keep for roi in mat_ret['s_labels']])[0]
    xyz1, labels1 = mat_ret['s_xyz'][idx_R], mat_ret['s_labels'][idx_R]
    su_codes1 = mat_ret['su_codes'][idx_R]
    mask1 = np.load(path_npz_ret+'masks_stat/'+mask_form.format(freq))
    mask1 = np.asarray([not x for x in mask1])
    
    #select only electrodes present both during encoding and retrieval
    to_keep_enc = np.where((xyz0==xyz1[:,None]).all(-1))[1]
    to_keep_ret = np.where((xyz1==xyz0[:,None]).all(-1))[1]
    mask0_new, mask1_new = mask0[to_keep_enc], mask1[to_keep_ret]
    print(len(mask0_new),len(mask1_new))
    
    xyz_shared = xyz0[to_keep_enc]
    xyz0_sig = xyz0[to_keep_enc][mask0_new]
    xyz1_sig = xyz1[to_keep_ret][mask1_new]
    print(freq,xyz0_sig.shape,xyz1_sig.shape)
            
    #create an array of region labels for encoding and retrieval
    dict_regions ={ 'ACC':'Frontal','IFG':'Frontal','MFG':'Frontal','SFG':'Frontal',
                'Amg':'Olf','pPirT':'Olf','Amg-PirT':'Olf','Ins':'Olf','OFC':'Olf',
                'HC':'MTL','PHG':'MTL'}
    regions0 = np.vectorize(dict_regions.get)(labels0[to_keep_enc])
        
    #loop on all elecs and create a code indicating when it decodes
    codes_elecs, region_elecs, su_codes = [], [], []
    for i, elec in enumerate(xyz_shared):
        in_1 = np.where(np.all(elec == xyz0_sig, 1))[0] #encoding
        in_2 = np.where(np.all(elec == xyz1_sig, 1))[0] #retrieval
        
#         if in_1.size : #decode in encoding
#             codes_elecs.append(0)
#             su_codes.append(su_codes0[i])
#             region_elecs.append(regions0[i])
#         elif in_2.size: #decode in retrieval
#             su_codes.append(su_codes0[i])
#             codes_elecs.append(1)
#             region_elecs.append(regions0[i])
        if in_1.size and in_2.size: #decode in both
            codes_elecs.append(2)
            su_codes.append(su_codes0[i])
            region_elecs.append(regions0[i])
        elif in_1.size and not in_2.size: #decode only in encoding
            codes_elecs.append(0)
            su_codes.append(su_codes0[i])
            region_elecs.append(regions0[i])
        elif not in_1.size and in_2.size: #decode only in retrieval
            codes_elecs.append(1)
            su_codes.append(su_codes0[i])
            region_elecs.append(regions0[i])
    
    codes_elecs = np.asarray(codes_elecs)[np.newaxis]
    su_codes = np.asarray(su_codes)[np.newaxis]
    region_elecs = np.asarray(region_elecs)[np.newaxis]
    count = np.ones(shape=region_elecs.shape)
    print(freq,'regions',su_codes.shape,region_elecs.shape,codes_elecs.shape, count.shape)
    concat = np.concatenate((su_codes,codes_elecs,region_elecs,count),axis=0).swapaxes(0,1)
    df = pd.DataFrame(concat,columns=['su_codes','codes_elecs','region_elecs','count'])
    dfname = 'counting_E_R_elecs_{}_regions.csv'.format(freq)
    df.to_csv(path2save+dfname,index=False)    

230 230
0_theta (81, 3) (45, 3)
0_theta regions (1, 112) (1, 112) (1, 112) (1, 112)
230 230
1_alpha (66, 3) (43, 3)
1_alpha regions (1, 94) (1, 94) (1, 94) (1, 94)
230 230
2_beta (88, 3) (43, 3)
2_beta regions (1, 115) (1, 115) (1, 115) (1, 115)
230 230
3_gamma (141, 3) (44, 3)
3_gamma regions (1, 154) (1, 154) (1, 154) (1, 154)


In [3]:
Tvals, pvals = [], []
for freq in freqs:
    df = pd.read_csv(path2save+'counting_E_R_elecs_{}_regions.csv'.format(freq))
    print(df.columns)
    0/0
    df_gr = df.groupby(['su_codes','codes_elecs']).count().unstack()
    #print(df_gr)
    var0 = np.asarray([x for x in df_gr.iloc[:,0].values if str(x) != 'nan'])[:,np.newaxis]
    var1 = np.asarray([x for x in df_gr.iloc[:,1].values if str(x) != 'nan'])[:,np.newaxis]
    var2 = np.asarray([x for x in df_gr.iloc[:,2].values if str(x) != 'nan'])[:,np.newaxis]
    print(var0.shape,var1.shape,var2.shape)
    T0, p0 = ttest_perm_unpaired(var0,var1,n_perm=95)
    Tvals.append(T0)
    pvals.append(p0)
    T0, p0 = ttest_perm_unpaired(var0,var2,n_perm=95)
    Tvals.append(T0)
    pvals.append(p0)
    T0, p0 = ttest_perm_unpaired(var2,var1,n_perm=95)
    Tvals.append(T0)
    pvals.append(p0)
_,pvals_c = fdr_correction(pvals,alpha=0.05) 
print('all elecs E vs R',Tvals,pvals_c)

Tvals, pvals = [], []
for freq in freqs:
    df = pd.read_csv(path2save+'counting_E_R_elecs_{}_regions.csv'.format(freq))
    df_gr = df.groupby(['su_codes','codes_elecs','region_elecs']).count().unstack()
    for roi in df_gr.columns.get_level_values(1):
        df_roi = df_gr.iloc[:, df_gr.columns.get_level_values(1)==roi].unstack()
        var0 = [x for x in df_roi.iloc[:,0].values if str(x) != 'nan']
        var1 = [x for x in df_roi.iloc[:,1].values if str(x) != 'nan']
        var2 = [x for x in df_gr.iloc[:,2].values if str(x) != 'nan']
        T0, p0 = ttest_ind(var0,var1,equal_var=True)
        print(freq,roi,T0,p0)
        T0, p0 = ttest_ind(var2,var1,equal_var=True)
        print(freq,roi,T0,p0)
        T0, p0 = ttest_ind(var0,var2,equal_var=True)
        print(freq,roi,T0,p0)

Index(['su_codes', 'codes_elecs', 'region_elecs', 'count'], dtype='object')


ZeroDivisionError: division by zero

In [9]:
from scipy.stats import ttest_ind
from mne.stats import fdr_correction
Tvals, pvals = [], []
for freq in freqs:
    df = pd.read_csv(path2save+'counting_E_R_elecs_{}_regions.csv'.format(freq))
    #df = pd.read_csv(path2save+'counting_elecs_{}_regions.csv'.format(freq))
    df_gr = df.groupby(['su_codes','codes_elecs']).count().unstack()
    var0 = np.asarray([x for x in df_gr.iloc[:,0].values if str(x) != 'nan'])[:,np.newaxis]
    var1 = np.asarray([x for x in df_gr.iloc[:,1].values if str(x) != 'nan'])[:,np.newaxis]
    T0, p0 = ttest_ind(var0,var1,equal_var=False)
    Tvals.append(T0)
    pvals.append(p0)
_,pvals_c = fdr_correction(pvals,alpha=0.05) 
print('all elecs E vs R',Tvals,pvals)

Tvals, pvals = [], []
for freq in freqs:
    #df = pd.read_csv(path2save+'counting_elecs_{}_regions.csv'.format(freq))
    df = pd.read_csv(path2save+'counting_E_R_elecs_{}_regions.csv'.format(freq))
    df_gr = df.groupby(['su_codes','codes_elecs','region_elecs']).count().unstack()
    for roi in df_gr.columns.get_level_values(1):
        df_roi = df_gr.iloc[:, df_gr.columns.get_level_values(1)==roi].unstack()
        var0 = np.asarray([x for x in df_gr.iloc[:,0].values if str(x) != 'nan'])[:,np.newaxis]
        var1 = np.asarray([x for x in df_gr.iloc[:,1].values if str(x) != 'nan'])[:,np.newaxis]
        T0, p0 = ttest_ind(var0,var1,equal_var=False)
        print(freq, roi, T0,p0)
Tvals = [x for x in Tvals if str(x) != 'nan']
pvals = [x for x in pvals if str(x) != 'nan']
print(pvals)
_,pvals_c = fdr_correction(pvals,alpha=0.05) 
print('all elecs E vs R',Tvals,pvals_c)


all elecs E vs R [array([ 1.82668175]), array([ 1.26649217]), array([ 2.68830443]), array([ 3.28310404])] [array([ 0.11838849]), array([ 0.24748837]), array([ 0.02394478]), array([ 0.02071291])]
0_theta Frontal [ 1.35969594] [ 0.1939519]
0_theta MTL [ 1.35969594] [ 0.1939519]
0_theta Olf [ 1.35969594] [ 0.1939519]
1_alpha Frontal [ 0.44677149] [ 0.66259823]
1_alpha MTL [ 0.44677149] [ 0.66259823]
1_alpha Olf [ 0.44677149] [ 0.66259823]
2_beta Frontal [ 0.74682934] [ 0.46401418]
2_beta MTL [ 0.74682934] [ 0.46401418]
2_beta Olf [ 0.74682934] [ 0.46401418]
3_gamma Frontal [ 1.52123909] [ 0.15047518]
3_gamma MTL [ 1.52123909] [ 0.15047518]
3_gamma Olf [ 1.52123909] [ 0.15047518]
[]
all elecs E vs R [] []
